# Chapter 3. 숫자 세계로 떠난 자연어  

말뭉치의 의미와 문맥을 학습한 언어 모델을 활용해 문서 분류, 개체명 인식 등 각종 태스크를 수행할 수 있습니다. 이 언어 모델은 세부 태스크의 성능을 결정하는데요, 요즘에는 트랜스포머 기반의 언어 모델이 각광받고 있습니다. 이 장에서는 언어 모델이 말뭉치의 어떤 의미 정보를 학습하는지, 트랜스포머의 핵심 동작 원리는 무엇인지, 트랜스포머가 기본 뼈대인 BERT와 GPT 모델의 특징 등을 살펴보겠습니다.

## <3-1 미리 학습된 언어 모델>  

최근 BERT, GPT 같은 모델이 주목받게 된 이유는 성능 때문입니다. 이 모델들을 사용하면 문서 분류, 개체명 인식 등 어떤 태스크든지 점수가 이전과 비교해 큰 폭으로 오르기 때문인데요. BERT, GPT 따위의 부류는 프리트레인(미리 학습된) 언어 모델이라는 공통점이 있습니다.  
이 절에서는 언어 모델의 개념과 프리트레인의 종류, 프리트레인을 마친 언어 모델이 성능이 좋은 이유 등을 살펴보고자 합니다.

### 언어 모델
---
***언어 모델(language model)*** 이란 단어 시퀀스에 확률을 부여하는 모델입니다. 다시 말해 단어 시퀀스를 입력받아 해당 시퀀스가 얼마나 그럴듯한지 확률을 출력하는 모델입니다. 따라서 한국어 말뭉치로 학습한 언어 모델은 자연스러운 한국어 문장에 높은 확률값을 부여합니다. 어떤 문장이 한국어스러운지 해당 모델이 이해하고 있다는 것이죠.  


문장에서 $i$번째로 등장하는 단어를 $w_{i}$ 로 표시한다면 $n$ 개 단어로 구성된 문장이 해당 언어에서 등장할 확률, 즉 언어 모델의 출력은 다음 수식처럼 쓸 수 있습니다. 이 수식은 $n$ 개 단어가 동시에 나타날 ***결합 확률(joint probability)*** 을 의미합니다. 잘 학습된 한국어 모델이 있다면 $P(무모, 운전)$ 보다는 $P(난폭, 운전)$ 이 큰 확률값을 지닐 겁니다.

<center><수식 3-1 언어모델></center>  
$$P(w_1, w_2, w_3, w_4, ... , w_n)$$

그렇다면 ***난폭*** 이 나타난 다음에 ***운전*** 이 나타날 확률은 어떻게 정의할까요? 이러한 확률을 ***조건부 확률(conditional probability)*** 이라고 하는데요,  
다음 수식과 같이 정의됩니다.

<center><수식 3-2 '난폭' 다음에 '운전'이 올 조건부 확률></center>  
$$P(운전|난폭) = \frac{P(난폭,운전)}{P(난폭)}$$

조건부 확률(위 수식 좌변)을 표기할 때 결과가 되는 사건(**운전**)을 앞에, 조건이 되는 사건(**난폭**)은 뒤에 씁니다. 조건이 되는 사건이 우변 분자의 일부, 그리고 우변 분모를 구성하고 있음을 볼 수 있는데요, 이는 결과가 되는 사건(**운전**)은 조건이 되는 사건(**난폭**)의 영향을 받아 변한다는 개념을 내포하고 있습니다. 그도 그럴 것이 앞선 단어가 **난폭**이라면 다음 단어로 어떤 것이 자연스러울지 그 선택지가 확 줄어들겠죠.  

결합 확률과 조건부 확률 사이에는 밀접한 관련이 있습니다. 조건부 확률의 정의에 따라 단어 3개가 동시에 등장할 결합 확률을 수식으로 나타내면 다음과 같습니다.

<center><수식 3-3 결합 확률과 조건부 확률 사이의 관계></center>  
$$P(w_1, w_2, w_3) = P(w_1)xP(w_2|w_1)xP(w_3|w_1, w_2)$$

수식 3-2 조건부 확률의 정의에 따라 수식 3-3의 우변을 쭉 펼쳐 계산해 보면 좌변과 같습니다. 이를 직관적으로 곱씹어 보면, 단어 3개로 구성된 문장이 나타나려면(즉, 단어 3개가 동시에 등장하려면) 다음 3가지 ***사건(event)*** 이 동시에 일어나야 한다는 말이 됩니다.

- 첫 번째 단어($w_1$) 등장
- 첫 번째 단어 등장($w_1$) 후 두 번째 단어($w_2$) 등장
- 첫 번째 단어($w_1$)와 두 번째 단어 등장($w_2$) 후 세 번째 단어($w_3$) 등장

이로부터 수식 3-1의 언어 모델을 조건부 확률 개념으로 다시 쓰면 다음 수식과 같습니다. 요약하면 **전체 단어 시퀀스가 나타날 확률**(다음 수식 좌변)**은 이전 단어들이 주어졌을 때 다음 단어가 등장할 확률의 연쇄**(다음 수식 우변)와 같다는 이야기입니다. 이 때문에 언어 모델을 **이전 단어들이 주어졌을 때 다음 단어가 나타날 확률을 부여하는 모델**이라고 정의하기도 합니다.

<center><수식 3-4 조건부 확률로 다시 쓴 언어 모델></center>  
$$P(w_1, w_2, w_3, w_4, ... , w_n) = \prod_{k=1}^n P(w_i|w_1,..., w_{i-1})$$
